<a href="https://colab.research.google.com/github/FunNyLuAz/Performance-em-Sistemas-Ciberfisicos/blob/main/Atividade%20Avaliativa%20Em%20Grupo%20RA3-1/Atividade%20Avaliativa%20Em%20Grupo%20RA3-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Condição de Corrida e Seção Crítica
Em sistemas de computação, a condição de corrida ocorre quando duas ou mais operações devem ocorrer na mesma ordem, mas a ordem em que as operações são executadas não é controlada pelo programa, o que resulta em comportamentos inesperados e/ou indeterminados. Estas condições são particularmente comuns in sistemas paralelos e concorrentes, como sistemas distribuídos e sistemas multithread.

Devido a sua natureza não determinística é uma situação difícil de prever. A condição de corrida é frequentemente considerada um dos problemas mais difíceis na programação concorrente. Estas condições podem causar problemas, como falhas de segurança, corrupção de dados e comportamentos de sistemas erráticos.

Para obter uma melhor compreensão das condições de corrida, vamos examinar un exemplo clássico de um sistema multithread: Imagine que temos duas threads, A e B, que precisam incrementar o valor armazenado em uma variável compartilhada.

A operação de incremento normalmente é composta por três passos:

1. Ler o valor atual da variável (por exemplo, counter é 0)
2. Incrementar o valor (por exemplo, 0 torna-se 1)
3. Gravar o valor incrementado de volta à variável

No entanto, em um sistema multithread sem controle adequado de acesso, a sequência dessas operações pode ser interrompida, como abaixo:

1. A thread A lê o valor atual do counter (0)
2. A thread B lê o valor atual do counter (ainda 0, pois A ainda não incrementou o valor)
3. A thread A incrementa o valor e grava o valor incrementado de volta (1)
4. A thread B, que ainda pensa que o valor é 0, incrementa o valor e grava o valor incrementado de volta (1)

Nesta situação, embora A e B tenham tentado aumentar a contagem, a contagem é apenas 1 em vez de 2. Isso é uma situação de competição entre os threads e é um bom exemplo de uma condição de corrida.

Para evitar condições de corrida, os programadores usam várias técnicas de controle de concorrência, como bloqueios (por exemplo, mutexes ou semáforos), que garantem que certas operações sejam atômicas, ou seja, indivisíveis. No contexto do exemplo acima, um semáforo pode ser usado para garantir que apenas uma thread possa realizar a operação de incremento de uma vez. Desta forma, a sequência de ler, incrementar e gravar é sempre concluída por uma thread antes que a outra comece, evitando a condição de corrida.

Em termos mais gerais, o conceito de 'seção crítica' é usado para denotar um bloco de código que acessa recursos compartilhados e que não deve ser simultaneamente executado por mais de um thread. As técnicas de controle de concorrência visam garantir que apenas um thread possa entrar em sua seção crítica de cada vez.


## Vamos ver um Exemplo

In [ ]:
import threading

# um objeto compartilhado
counter = 0

# função que será executada pelas threads
def increment_counter(thread_name):
    global counter
    for _ in range(5):  # reduzindo o número de operações para simplificar a visualização
        print(f"{thread_name} está incrementando o contador.")
        counter += 1
        print(f"{thread_name} incrementou o contador. Valor do contador: {counter}")

# criação de 3 threads
threads = []
for i in range(3):
    t = threading.Thread(target=increment_counter, args=(f"Thread {i + 1}",))
    threads.append(t)
    t.start()

# espera todas as threads terminarem
for t in threads:
    t.join()

print(f"\nValor final do contador: {counter}")
print("Valor esperado do contador: 15")  # 3 threads * 5 incrementos cada = 15


Thread 1 está incrementando o contador.
Thread 1 incrementou o contador. Valor do contador: 1
Thread 1 está incrementando o contador.
Thread 1 incrementou o contador. Valor do contador: 2
Thread 1 está incrementando o contador.
Thread 1 incrementou o contador. Valor do contador: 3
Thread 1 está incrementando o contador.
Thread 1 incrementou o contador. Valor do contador: 4
Thread 1 está incrementando o contador.
Thread 1 incrementou o contador. Valor do contador: 5
Thread 2 está incrementando o contador.
Thread 2 incrementou o contador. Valor do contador: 6
Thread 2 está incrementando o contador.
Thread 2 incrementou o contador. Valor do contador: 7
Thread 2 está incrementando o contador.
Thread 2 incrementou o contador. Valor do contador: 8
Thread 2 está incrementando o contador.
Thread 2 incrementou o contador. Valor do contador: 9
Thread 2 está incrementando o contador.
Thread 2 incrementou o contador. Valor do contador: 10
Thread 3 está incrementando o contador.
Thread 3 incremento

## O problema
No código acima deveríamos ter uma condição de corrida, não há nada que garanta que um thread não vá sobrescrever o valor do outro. Contudo, é python, então o GIL  (Global Interpreter Lock) não deixa o código CPU-Bond ser executado em paralelo, logo não há sobreposição de escrita no objeto compartilhado (Global) counter. 

In [ ]:
import threading
import time

# Inicializando o objeto compartilhado
counter = 0

def increment_counter(thread_name):
    global counter
    for _ in range(3):
        # Cada thread lê o valor atual do contador
        print(f"{thread_name} está lendo o contador.")
        value = counter

        # Indicando que a thread vai incrementar o contador
        print(f"{thread_name} incrementará o contador.")
        # Simulando algum atraso que pode permitir que outras threads leiam o contador antes deste incremento
        time.sleep(0.1)
        
        # Incrementando o contador
        counter = value + 1
        print(f"{thread_name} incrementou o contador. Valor do contador: {counter}")

# Criando as threads
threads = []
for i in range(3):
    # Criando uma nova thread
    t = threading.Thread(target=increment_counter, args=(f"Thread {i + 1}",))
    threads.append(t)
    t.start()  # Iniciando a thread

# Esperando todas as threads terminarem antes de mostrar o valor final do contador
for t in threads:
    t.join()

print(f"\nValor final do contador: {counter}")
print("Valor esperado do contador: 9")  # 3 threads * 3 incrementos cada = 9


Thread 1 está lendo o contador.Thread 2 está lendo o contador.
Thread 2 incrementará o contador.

Thread 1 incrementará o contador.
Thread 3 está lendo o contador.
Thread 3 incrementará o contador.
Thread 2 incrementou o contador. Valor do contador: 1
Thread 2 está lendo o contador.
Thread 2 incrementará o contador.
Thread 1 incrementou o contador. Valor do contador: 1
Thread 1 está lendo o contador.
Thread 1 incrementará o contador.
Thread 3 incrementou o contador. Valor do contador: 1
Thread 3 está lendo o contador.
Thread 3 incrementará o contador.
Thread 2 incrementou o contador. Valor do contador: 2
Thread 2 está lendo o contador.
Thread 2 incrementará o contador.
Thread 1 incrementou o contador. Valor do contador: 2
Thread 1 está lendo o contador.
Thread 1 incrementará o contador.
Thread 3 incrementou o contador. Valor do contador: 2
Thread 3 está lendo o contador.
Thread 3 incrementará o contador.
Thread 2 incrementou o contador. Valor do contador: 3
Thread 1 incrementou o conta

## solucionando nada.
Como não há problema, por causa do GIL, a criação do semáforo é meramente didática. 

## Agora vamos forçar um problema de condição de corrida
Na maioria das linguagens, a condição de corrida é mais propensa a ocorrer quando temos uma operação que envolve mais de um passo e que é interrompida entre esses passos. Vamos tentar dividindo o incremento em duas etapas: uma para leitura e outra para gravação.

In [ ]:
import threading
import time

counter = 0

def increment_counter(thread_name):
    global counter
    for _ in range(3):
        print(f"{thread_name} está lendo o contador.")
        value = counter
        print(f"{thread_name} incrementará o contador.")
        time.sleep(0.1)  # simulando algum atraso
        counter = value + 1
        print(f"{thread_name} incrementou o contador. Valor do contador: {counter}")

threads = []
for i in range(3):
    t = threading.Thread(target=increment_counter, args=(f"Thread {i + 1}",))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print(f"\nValor final do contador: {counter}")
print("Valor esperado do contador: 9")  # 3 threads * 3 incrementos cada = 9


Thread 1 está lendo o contador.
Thread 1 incrementará o contador.
Thread 2 está lendo o contador.
Thread 2 incrementará o contador.
Thread 3 está lendo o contador.
Thread 3 incrementará o contador.
Thread 1 incrementou o contador. Valor do contador: 1
Thread 1 está lendo o contador.
Thread 1 incrementará o contador.
Thread 2 incrementou o contador. Valor do contador: 1
Thread 2 está lendo o contador.
Thread 2 incrementará o contador.
Thread 3 incrementou o contador. Valor do contador: 1
Thread 3 está lendo o contador.
Thread 3 incrementará o contador.
Thread 1 incrementou o contador. Valor do contador: 2
Thread 1 está lendo o contador.
Thread 1 incrementará o contador.
Thread 2 incrementou o contador. Valor do contador: 2
Thread 2 está lendo o contador.
Thread 2 incrementará o contador.
Thread 3 incrementou o contador. Valor do contador: 2
Thread 3 está lendo o contador.
Thread 3 incrementará o contador.
Thread 1 incrementou o contador. Valor do contador: 3
Thread 2 incrementou o conta

## Finalmente, o problema
Neste exemplo, conseguimos provocar uma condição de corrida. Novamente, só para lembrar, se fosse o java, ou o C++, talvez não precisássemos dividir a tarefa do thread em dois tempos. Como não há nenhuma restrição no compilador destas duas linguagens (ou interpretador) que é o caso do java, então existe uma chance de que dois threads tentem escrever ao mesmo tempo. 

## Agora vamos corrigir de verdade. 
Vou criar o semáforo, ativar e desativar nos lugares certos para garantir que não exista a condição de corrida. 

In [ ]:
import threading
import time

# Inicializando o objeto compartilhado
counter = 0
# Criando um semáforo. Um semáforo é usado para controlar o acesso a um recurso comum.
semaphore = threading.Semaphore()

def increment_counter(thread_name):
    global counter
    for _ in range(3):
        # Cada thread deve adquirir o semáforo antes de poder ler/modificar o contador
        semaphore.acquire()
        
        print(f"{thread_name} está lendo o contador.")
        value = counter
        
        print(f"{thread_name} incrementará o contador.")
        time.sleep(0.1)  # Simulando algum atraso
        
        counter = value + 1
        print(f"{thread_name} incrementou o contador. Valor do contador: {counter}")
        
        # Depois que a thread terminou de modificar o contador, ela deve liberar o semáforo.
        # Isso permite que outras threads adquiram o semáforo e modifiquem o contador.
        semaphore.release()

# Criando as threads
threads = []
for i in range(3):
    # Criando uma nova thread
    t = threading.Thread(target=increment_counter, args=(f"Thread {i + 1}",))
    threads.append(t)
    t.start()  # Iniciando a thread

# Esperando todas as threads terminarem antes de mostrar o valor final do contador
for t in threads:
    t.join()

print(f"\nValor final do contador: {counter}")
print("Valor esperado do contador: 9")  # 3 threads * 3 incrementos cada = 9


Thread 1 está lendo o contador.
Thread 1 incrementará o contador.
Thread 1 incrementou o contador. Valor do contador: 1
Thread 1 está lendo o contador.
Thread 1 incrementará o contador.
Thread 1 incrementou o contador. Valor do contador: 2
Thread 1 está lendo o contador.
Thread 1 incrementará o contador.
Thread 1 incrementou o contador. Valor do contador: 3
Thread 2 está lendo o contador.
Thread 2 incrementará o contador.
Thread 2 incrementou o contador. Valor do contador: 4
Thread 2 está lendo o contador.
Thread 2 incrementará o contador.
Thread 2 incrementou o contador. Valor do contador: 5
Thread 2 está lendo o contador.
Thread 2 incrementará o contador.
Thread 2 incrementou o contador. Valor do contador: 6
Thread 3 está lendo o contador.
Thread 3 incrementará o contador.
Thread 3 incrementou o contador. Valor do contador: 7
Thread 3 está lendo o contador.
Thread 3 incrementará o contador.
Thread 3 incrementou o contador. Valor do contador: 8
Thread 3 está lendo o contador.
Thread 3

## Vamos ver um exemplo um tanto mais complexo.

In [ ]:
# Importando os módulos necessários
import threading
import time

# Seções críticas do nosso programa
# A lista de inteiros e a variável float que serão manipuladas pelas threads
int_list = [0, 1, 2, 3, 4]
float_var = 1.0

# Semáforos para controlar o acesso às seções críticas
# Cada semáforo é inicializado com 1, o que significa que apenas uma thread pode adquirir cada semáforo de cada vez
int_list_semaphore = threading.Semaphore()
float_var_semaphore = threading.Semaphore()

def manipulate_data(thread_name):
    global int_list
    global float_var

    # Operações na lista de inteiros
    int_list_semaphore.acquire()  # Adquire o semáforo para a lista de inteiros
    print(f"{thread_name} está lendo a lista de inteiros: {int_list}")
    time.sleep(0.1)  # Espera um pouco para simular o trabalho
    int_list = [x + 1 for x in int_list]  # Incrementa cada item da lista
    print(f"{thread_name} incrementou a lista de inteiros: {int_list}")
    int_list_semaphore.release()  # Libera o semáforo para a lista de inteiros

    # Operações na variável float
    float_var_semaphore.acquire()  # Adquire o semáforo para a variável float
    print(f"{thread_name} está lendo a variável float: {float_var}")
    time.sleep(0.1)  # Espera um pouco para simular o trabalho
    float_var *= 2  # Duplica a variável float
    print(f"{thread_name} dobrou a variável float: {float_var}")
    float_var_semaphore.release()  # Libera o semáforo para a variável float

# Criação das threads
threads = []
for i in range(5):
    # Cria uma nova thread e define a função manipulate_data como o alvo da thread
    # Passa a string "Thread {i + 1}" como argumento para a função manipulate_data
    t = threading.Thread(target=manipulate_data, args=(f"Thread {i + 1}",))
    threads.append(t)  # Adiciona a thread recém-criada à lista de threads
    t.start()  # Inicia a execução da thread

# Espera todas as threads terminarem
for t in threads:
    t.join()

# Imprime os valores finais da lista de inteiros e da variável float
print(f"\nValor final da lista de inteiros: {int_list}")
print(f"Valor esperado da lista de inteiros: [5, 6, 7, 8, 9]")
print(f"Valor final da variável float: {float_var}")
print(f"Valor esperado da variável float: 32.0")


Thread 1 está lendo a lista de inteiros: [0, 1, 2, 3, 4]
Thread 1 incrementou a lista de inteiros: [1, 2, 3, 4, 5]
Thread 1 está lendo a variável float: 1.0
Thread 2 está lendo a lista de inteiros: [1, 2, 3, 4, 5]
Thread 1 dobrou a variável float: 2.0
Thread 2 incrementou a lista de inteiros: [2, 3, 4, 5, 6]
Thread 2 está lendo a variável float: 2.0Thread 3 está lendo a lista de inteiros: [2, 3, 4, 5, 6]

Thread 3 incrementou a lista de inteiros: [3, 4, 5, 6, 7]
Thread 2 dobrou a variável float: 4.0
Thread 4 está lendo a lista de inteiros: [3, 4, 5, 6, 7]
Thread 3 está lendo a variável float: 4.0
Thread 3 dobrou a variável float: 8.0
Thread 4 incrementou a lista de inteiros: [4, 5, 6, 7, 8]
Thread 4 está lendo a variável float: 8.0
Thread 5 está lendo a lista de inteiros: [4, 5, 6, 7, 8]
Thread 4 dobrou a variável float: 16.0
Thread 5 incrementou a lista de inteiros: [5, 6, 7, 8, 9]
Thread 5 está lendo a variável float: 16.0
Thread 5 dobrou a variável float: 32.0

Valor final da lista 

## Um exemplo de como usar a Beautifullsoap

In [ ]:


# Importando as bibliotecas necessárias
import requests
from bs4 import BeautifulSoup

# A URL da página da web que você quer acessar
url = "https://en.wikipedia.org/wiki/Race_condition#:~:text=A%20race%20condition%20can%20arise,bugs%20due%20to%20unanticipated%20behavior"

# Enviar uma requisição GET para a página da web
response = requests.get(url)

# Inicializar um objeto BeautifulSoup com o conteúdo da página
soup = BeautifulSoup(response.content, 'html.parser')

# Extrair todo o texto da página (sem tags HTML) e armazená-lo em uma variável
page_text = soup.get_text()

# Imprimir o texto da página
print(page_text)






Race condition - Wikipedia





































Jump to content








Main menu





Main menu
move to sidebar
hide



		Navigation
	

Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate




		Contribute
	

HelpLearn to editCommunity portalRecent changesUpload file




Languages

Language links are at the top of the page across from the title.



















Search















Create accountLog in






Personal tools




 Create account Log in




		Pages for logged out editors learn more


ContributionsTalk


























Contents
move to sidebar
hide




(Top)





1In electronics


				Toggle In electronics subsection
			




1.1Critical and non-critical forms







1.2Static, dynamic, and essential forms









2Workarounds







3In software


				Toggle In software subsection
			




3.1Example







3.2Data race







3.3Example definitions of data races in particular concurrency models





3.3.1Sequential 

# Atividade Prática Avaliativa Em Grupo - RA3-1
Para realizar esta atividade e conseguir os pontos equivalentes na média da RA-3 você deve fazer uma cópia deste notebook, manter inalterado este enunciado, incluir o nome de todos os componentes do grupo e, finalmente resolver o exercício a seguir em uma célula de código: 

Seu objetivo é escrever um programa Python que recupere o texto de cinco páginas diferentes da Wikipedia. Cada página deve ser recuperada por uma thread diferente.

O programa deve cumprir os seguintes requisitos:

1. Use a biblioteca requests para realizar requisições HTTP GET para as páginas da Wikipedia.
2. Use a biblioteca BeautifulSoup para extrair o texto das páginas da web.
3. Crie uma thread para cada página da Wikipedia que você está acessando.
4. Cada thread deve escrever o texto extraído no mesmo arquivo de texto compartilhado.
5. Garanta que não ocorram condições de corrida ao escrever no arquivo de texto compartilhado.
6. O thread que gravou cada texto no arquivo deve ser identificado no próprio texto gravado e em uma impressão no terminal que permita acompanhar o processo de gravação. 

Lembre-se não podem haver condições de corrida. Use semáforos. 


Lucas Azevedo Dias

In [12]:
from bs4 import BeautifulSoup
from io import BufferedWriter
from requests import get
from threading import Thread, Semaphore


filename = "page_text.txt"
file_semaphore = Semaphore()


def thread_target(id:int, file:BufferedWriter):
    # Getting the page text of its url
    print(f"Thread {id + 1} started downloading")
    response = get(urls[id])
    page_text = BeautifulSoup(response.content, 'html.parser').get_text().replace("\n\n", "")
    print(f"Thread {id + 1} downloaded the file successfully")

    # Writing page text in file
    file_semaphore.acquire()
    print(f"Thread {id + 1} got the file")
    file.write(f"###THREAD{id + 1} START{'$' * 50}\n")
    file.write(page_text[:page_text.find("\n")] + "\n")
    file.write(f"###THREAD{id + 1} END{'$' * 50}\n")
    print(f"Thread {id + 1} freed the file")
    file_semaphore.release()


if __name__ == "__main__":
    urls = [
        "https://pt.wikipedia.org/wiki/Brasil",
        "https://pt.wikipedia.org/wiki/Fran%C3%A7a",
        "https://pt.wikipedia.org/wiki/J%C3%BAlio_C%C3%A9sar",
        "https://pt.wikipedia.org/wiki/Roma",
        "https://pt.wikipedia.org/wiki/Portugal"
    ]

    open(filename, "w", encoding="utf-8").close()  # Erases the old file content
    file = open(filename, "a", encoding="utf-8")

    threads = []
    for i in range(len(urls)):
        threads.append(Thread(target=thread_target, args=(i, file)))
    
    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()

    file.close()

    # Prints the output file
    print("\nPrinting output file...")
    with open(filename, "r", encoding="utf-8") as file:
      print(file.read())
      file.close()

Thread 1 started downloading
Thread 2 started downloading
Thread 3 started downloading
Thread 4 started downloadingThread 5 started downloading

Thread 3 downloaded the file successfully
Thread 3 got the file
Thread 3 freed the file
Thread 4 downloaded the file successfully
Thread 4 got the file
Thread 4 freed the file
Thread 2 downloaded the file successfully
Thread 2 got the file
Thread 2 freed the file
Thread 5 downloaded the file successfully
Thread 5 got the file
Thread 5 freed the file
Thread 1 downloaded the file successfully
Thread 1 got the file
Thread 1 freed the file

Printing output file...
###THREAD3 START$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Júlio César – Wikipédia, a enciclopédia livre
###THREAD3 END$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
###THREAD4 START$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Roma – Wikipédia, a enciclopédia livre
###THREAD4 END$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
###THREAD2 START$$$$$$$$$$$$$$$$$$$$$$$$$